In [103]:
from mpi4py import MPI
import timeit
import numpy as np
COMM = MPI.COMM_WORLD
SIZE = COMM.Get_size()

RANK = COMM.Get_rank()

from manapy.partitions import MeshPartition
from manapy.ddm import Domain

from manapy.base.base import Struct
from manapy.ast import Variable
from manapy.base.base import Struct
import os

start = timeit.default_timer()

#get the mesh directory

 
MESH_DIR = "/home/asus/manapy/mesh"
filename = "choc2_ref.msh"

filename = os.path.join(MESH_DIR, filename)
dim = 2

running_conf = Struct(backend="numba", signature=True, cache=True, float_precision="double")#, int_precision="signed")
mesh = MeshPartition(filename, dim=dim, conf=running_conf, periodic=[0,0,0])

(int32[:,:], int32[:,:])
Reading gmsh file ...
Saving partition files ...
Number of Cells: 4084082
Number of Vertices: 2044900


In [104]:

domain = Domain(dim=dim, conf=running_conf)
faces = domain.faces
cells = domain.cells
halos = domain.halos
nodes = domain.nodes

nbnodes = domain.nbnodes
nbfaces = domain.nbfaces
nbcells = domain.nbcells


Local domain contruction ...
(int32[:,:], float64[:,:], int32, float64[:,:], float64[:])
(int32[:,:], float64[:,:], int32, int32)
(int32[:,:], int32, int32[:,:], int32[:,:])
(int32, int64[:], int32[:,:], int32[:,:], int32)
(int32[:,:], int32, int32, int32[:,:], int32)
(int32[:,:], int32[:,:], int32, int32)
(int32[:,:], int32[:,:], int32[:], float64[:,:], float64[:,:], int32, float64[:,:], float64[:], float64[:,:], int32[:], float64[:,:])
(float64[:,:], float64[:,:], int32[:,:], float64[:,:], int32, float64[:,:,:], int32)
(int32[:,:], int32[:,:], int32[:,:], int32[:], int32[:], int32[:], int32, int32, int32, int32)
(float64[:,:,:], int32[:,:])
(float64[:,:], int32[:,:], int32[:,:], int32[:,:], int32[:,:], int32[:,:], float64[:,:], float64[:,:], float64[:,:], float64[:,:], float64[:], float64[:], float64[:], float64[:], int32[:], float64[:,:])
(int32[:,:], int32[:,:], float64[:,:], int32[:], float64[:,:], float64[:,:], float64[:,:], int32[:], float64[:,:], float64[:], float64[:], float64

In [105]:

boundaries = {"in" : "neumann",
              "out" : "neumann",
              "upper":"neumann",
              "bottom":"neumann"
              }

boundaries_rho = {"in" : "dirichlet",
              "out" : "dirichlet",
              "upper":"neumann",
              "bottom":"neumann"
              }

values_rho = {"in" : 1, "out" :0.125
          
          }
values_p={"in" : 1, "out" :0.1
          
          }

rho   = Variable(domain=domain,BC=boundaries_rho,values=values_rho)
uv  = Variable(domain=domain, BC=boundaries)
v  = Variable(domain=domain, BC=boundaries)
P     = Variable(domain=domain,BC=boundaries_rho,values=values_p)
rhoE  = Variable(domain=domain,BC=boundaries)
E=Variable(domain=domain,BC=boundaries)
e_inernal=Variable(domain=domain,BC=boundaries)

In [106]:
def initialisation_tubeChok2(rho:'float[:]', uv:'float[:]', v:'float[:]', rhoE:'float[:]', P:'float[:]',E:'float[:]',e_inernal:'float[:]', center:'float[:,:]', gamma='float'):
   
    nbelements = len(center)
    
    for i in range(nbelements):
        xcent = center[i][0]
        #print(xcent)
        if xcent< 0.5:
            rho[i]=1
            uv[i]=0
            v[i]=0
            P[i]=1
            rhoE[i]=P[i]/(gamma-1) +0.5*(uv[i]**2+v[i]**2)
            E[i]=rhoE[i]/rho[i]
            e_inernal[i]=P[i]/((gamma-1)*rho[i])
        else:
            rho[i]=0.125
            uv[i]=0
            v[i]=0
            P[i]=0.1
            rhoE[i]=P[i]/(gamma-1) +0.5*(uv[i]**2+v[i]**2)
            E[i]=rhoE[i]/rho[i]
            e_inernal[i]=P[i]/((gamma-1)*rho[i])


In [107]:

initialisation_tubeChok2(rho.cell, uv.cell, v.cell, rhoE.cell,  P.cell,E.cell,e_inernal.cell,cells.center, gamma=1.4)

In [108]:
# exacte solution
import numpy as np
import matplotlib.pyplot as plt

def f(p, alpha, pl, pr, al, ar, rl, rr, gamma):
    if alpha == 'L':
        p_alpha = pl
        a_alpha = al
        A_alpha = 2. / (rl * (gamma + 1.))
        B_alpha = (gamma - 1.) / (gamma + 1.) * pl
    elif alpha == 'R':
        p_alpha = pr
        a_alpha = ar
        A_alpha = 2. / (rr * (gamma + 1.))
        B_alpha = (gamma - 1.) / (gamma + 1.) * pr
    else:
        raise ValueError("Invalid alpha value. Must be 'L' or 'R'")

    if p > p_alpha:
        return (p - p_alpha) * np.sqrt(A_alpha / (p + B_alpha))
    else:
        return 2. * a_alpha / (gamma - 1.) * ((p / p_alpha) ** ((gamma - 1.) / (2. * gamma)) - 1.)

def g(w, ul, ur, pl, pr, al, ar, rl, rr, gamma):
    return f(w, 'L', pl, pr, al, ar, rl, rr, gamma) + f(w, 'R', pl, pr, al, ar, rl, rr, gamma) + ur - ul

def dichotomie(bas, haut, ul, ur, pl, pr, al, ar, rl, rr, gamma):
    lower, upper = bas, haut
    while g(lower, ul, ur, pl, pr, al, ar, rl, rr, gamma) * g(upper, ul, ur, pl, pr, al, ar, rl, rr, gamma) > 0:
        upper = 2.0 * upper

    while upper - lower > 1e-8:
        middle = 0.5 * (upper + lower)
        if g(middle, ul, ur, pl, pr, al, ar, rl, rr, gamma) > 0:
            lower, upper = lower, middle
        else:
            lower, upper = middle, upper

    return 0.5 * (upper + lower)

def Exact_Euler(N, x, t, x_0, xmax, gamma, rl, rr, ul, ur, pl, pr):
    r = np.zeros(N)  # densite
    p = np.zeros(N)  # pression
    u = np.zeros(N)  # vitesse

    al = np.sqrt(gamma * pl / rl)
    ar = np.sqrt(gamma * pr / rr)
    p_star = dichotomie(0.0, max(pl, pr), ul, ur, pl, pr, al, ar, rl, rr, gamma)
    u_star = (ul + ur + f(p_star, 'R', pl, pr, al, ar, rl, rr, gamma) - f(p_star, 'L', pl, pr, al, ar, rl, rr, gamma)) * 0.5

    if p_star > pl:
        r_star_l = rl * (p_star / pl + (gamma - 1.) / (gamma + 1.)) / ((gamma - 1.) / (gamma + 1.) * p_star / pl + 1.)
    else:
        r_star_l = rl * (p_star / pl) ** (1. / gamma)

    a_star_l = al * (p_star / pl) ** ((gamma - 1.) / (2. * gamma))
    sigma_l = ul - al * np.sqrt((gamma + 1.) / (2. * gamma) * (p_star / pl) + (gamma - 1.) / (2. * gamma))
    sigma_H_l = ul - al
    sigma_T_l = u_star - a_star_l

    if p_star > pr:
        r_star_r = rr * (p_star / pr + (gamma - 1.) / (gamma + 1.)) / ((gamma - 1.) / (gamma + 1.) * (p_star / pr) + 1.)
    else:
        r_star_r = rr * (p_star / pr) ** (1. / gamma)

    a_star_r = ar * (p_star / pr) ** ((gamma - 1.) / (2. * gamma))
    sigma_r = ur + ar * np.sqrt((gamma + 1.) / (2. * gamma) * (p_star / pr) + (gamma - 1.) / (2. * gamma))
    sigma_H_r = ur + ar
    sigma_T_r = u_star + a_star_r

    for i in range(N):
        if p_star > pl:  # Gauche = choc
            if x[i] - xmax * x_0 < sigma_l * t:
                r[i], p[i], u[i] = rl, pl, ul
            elif sigma_l * t <= x[i] - xmax * x_0 <= u_star * t:
                r[i], p[i], u[i] = r_star_l, p_star, u_star
        else:  # Gauche = detente
            if x[i] - xmax * x_0 < sigma_H_l * t:
                r[i], p[i], u[i] = rl, pl, ul
            elif sigma_H_l * t <= x[i] - xmax * x_0 < sigma_T_l * t:
                r[i] = rl * (2. / (gamma + 1.) + (gamma - 1.) / ((gamma + 1.) * al) * (ul - (x[i] - x_0) / t)) ** (2. / (gamma - 1.))
                p[i] = pl * (2. / (gamma + 1.) + (gamma - 1.) / ((gamma + 1.) * al) * (ul - (x[i] - x_0) / t)) ** (2. * gamma / (gamma - 1.))
                u[i] = 2. / (gamma + 1.) * (al + (gamma - 1.) * 0.5 * ul + (x[i] - x_0) / t)
            elif sigma_T_l * t <= x[i] - xmax * x_0 <= u_star * t:
                r[i], p[i], u[i] = r_star_l, p_star, u_star

        if p_star > pr:  # Droite = choc
            if x[i] - xmax * x_0 > sigma_r * t:
                r[i], p[i], u[i] = rr, pr, ur
            elif u_star * t <= x[i] - xmax * x_0 <= sigma_r * t:
                r[i], p[i], u[i] = r_star_r, p_star, u_star
        else:  # Droite = detente
            if x[i] - xmax * x_0 > sigma_H_r * t:
                r[i], p[i], u[i] = rr, pr, ur
            elif sigma_T_r * t < x[i] - xmax * x_0 <= sigma_H_r * t:
                r[i] = rr * (2. / (gamma + 1.) - (gamma - 1.) / ((gamma + 1.) * ar) * (ur - (x[i] - x_0) / t)) ** (2. / (gamma - 1.))
                p[i] = pr * (2. / (gamma + 1.) - (gamma - 1.) / ((gamma + 1.) * ar) * (ur - (x[i] - x_0) / t)) ** (2. * gamma / (gamma - 1.))
                u[i] = 2. / (gamma + 1.) * (-ar + (gamma - 1.) * 0.5 * ur + (x[i] - x_0) / t)
            elif u_star * t <= x[i] - xmax * x_0 <= sigma_T_r * t:
                r[i], p[i], u[i] = r_star_r, p_star, u_star

    return r, u, p

# Example usage
gamma = 1.4
xmin, xmax = 0, 1
N = 1000
center=cells.center
nbelement=len(center)
#x=np.linspace(xmin,xmax,N)
x=[center[i][0]for i in range(nbelement)]

rl = 1.0
pl = 1.0
ul = 0
rr = 0.125
pr = 0.1
ur = 0.0
x_0 = 0.5
tf = 0.24

r, u, p = Exact_Euler(nbelement, x, tf, x_0, xmax, gamma, rl, rr, ul, ur, pl, pr)

In [109]:
for i in range(nbelement):
    rho.cell[i] = r[i]
    uv.cell[i]=  u[i] 
    e_inernal.cell[i] = p[i] / (r[i] * (gamma - 1.))
    E.cell[i]=  (0.5 * u[i]**2)/r[i] + p[i] / (r[i] * (gamma - 1.))
    P.cell[i]=p[i]

In [110]:
rho.update_ghost_value()     
uv.update_ghost_value()
E.update_ghost_value()
e_inernal.update_ghost_value()
P.update_ghost_value()

rho.interpolate_celltonode()  
uv.interpolate_celltonode()
E.interpolate_celltonode()
P.interpolate_celltonode()
e_inernal.interpolate_celltonode()

domain.save_on_node_multi(0, tf, 1, 1, variables=["rho", "u", "E","e_internal","p"],
                                      values=[rho.node, uv.node, E.node, e_inernal.node, P.node])

 **************************** Computing ****************************
$$$$$$$$$$$$$$$$$$$$$$$$$$$$ Saving Results $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Iteration =  1 time =  0.24 time step =  0
maxrho = 1.0000000000000002


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def sort_data(x, y):
    combined = list(zip(x, y))
    combined.sort(key=lambda pair: pair[0])  
    x_sorted, y_sorted = zip(*combined)
    return np.array(x_sorted), np.array(y_sorted)

plt.figure(figsize=(15, 5))

x_sorted, r_sorted = sort_data(x, r)
plt.subplot(1, 4, 1)  
plt.plot(x_sorted, r_sorted, 'b-')
plt.title(f'Rho, t={tf}')

x_sorted, p_sorted = sort_data(x, p)
plt.subplot(1, 4, 2)  
plt.plot(x_sorted, p_sorted, 'b-')
plt.title(f'P, t={tf}')

x_sorted, u_sorted = sort_data(x, u)
plt.subplot(1, 4, 3) 
plt.plot(x_sorted, u_sorted, 'b-')
plt.title(f'u, t={tf}')

x_sorted, u_sorted = sort_data(x, E.cell)
plt.subplot(1, 4, 4) 
plt.plot(x_sorted, u_sorted, 'b-')
plt.title(f'E, t={tf}')

plt.tight_layout()  
plt.show()



In [4]:
import pandas as pd

csv_file = "/home/asus/manapy/manapy/solution_refcsv/ref_allvar.csv"
data = pd.read_csv(csv_file)
data.head()

,rho,u,E,e_internal,p,Points:0,Points:1,Points:2
0,1.000,0.0,2.5,2.5,1.0,0.0,0.0000,0
1,0.125,0.0,2.0,2.0,0.1,1.0,0.0000,0
2,1.000,0.0,2.5,2.5,1.0,0.0,1.0000,0
3,0.125,0.0,2.0,2.0,0.1,1.0,1.0000,0
4,1.000,0.0,2.5,2.5,1.0,0.0,0.9993,0


In [1]:
import numpy as np
def compute_order(h1, h2, error1, error2):
        """Calcule l'ordre de convergence entre deux maillages."""
        return (np.log(error1) - np.log(error2)) / (np.log(h1) - np.log(h2))
